# CyberPuppy Integrated Gradients 可解釋性分析

本notebook使用Integrated Gradients方法為中文網路霸凌偵測模型提供可解釋性分析。

## 參考文獻
- [Captum官方文檔](https://captum.ai/)
- [Integrated Gradients論文](https://arxiv.org/abs/1703.01365)
- [BERT解釋性教程](https://captum.ai/tutorials/Bert_SQUAD_Interpret)
- [中文NLP可解釋性範例](https://github.com/pytorch/captum/blob/master/tutorials/)

In [ ]:
# 安裝必要套件
!pip install captum matplotlib seaborn pandas numpy torch transformers

In [ ]:
import sys
import os
sys.path.append('../')

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# 設定中文字體
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 忽略警告
warnings.filterwarnings('ignore')

# 本地imports
from src.cyberpuppy.models.baselines import BaselineModel, ModelConfig
from src.cyberpuppy.explain.ig import (
    IntegratedGradientsExplainer,
    BiasAnalyzer,
    create_attribution_heatmap,
    save_attribution_report
)
from src.cyberpuppy.labeling.label_map import UnifiedLabel, ToxicityLevel, EmotionType

print("環境設定完成")

## 1. 載入模型

載入已訓練的CyberPuppy基線模型

In [ ]:
# 設定模型路徑（請根據實際情況調整）
MODEL_PATH = "../models/baseline"  # 或其他已訓練模型路徑

# 檢查模型是否存在
if Path(MODEL_PATH).exists():
    print(f"載入模型: {MODEL_PATH}")
    model = BaselineModel.load_model(MODEL_PATH)
    print(f"模型載入成功: {model.config.model_name}")
else:
    print("模型不存在，使用示例配置創建模型（注意：未訓練）")
    config = ModelConfig(
        model_name="hfl/chinese-macbert-base",
        use_focal_loss=True
    )
    model = BaselineModel(config)
    print("警告：使用未訓練的模型，結果僅供演示")

# 設定設備
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

## 2. 初始化IG解釋器

基於Captum的Integrated Gradients實現

In [ ]:
# 初始化IG解釋器
explainer = IntegratedGradientsExplainer(model, device)
print("IG解釋器初始化完成")

# 初始化偏見分析器
bias_analyzer = BiasAnalyzer(explainer)
print("偏見分析器初始化完成")

## 3. 測試文本準備

準備不同類型的測試文本

In [ ]:
# 測試文本集合
test_texts = {
    "毒性辱罵": [
        "你這個垃圾，滾開！",
        "去死吧，沒用的廢物",
        "你真是個白痴，什麼都不懂"
    ],
    "威脅霸凌": [
        "我要殺了你這個混蛋",
        "別讓我再看到你，否則你會後悔",
        "你等著瞧，我不會放過你的"
    ],
    "身份攻擊": [
        "你們這些外地人就是素質差",
        "女生就是不適合做這種工作",
        "老年人就是跟不上時代"
    ],
    "正面中性": [
        "今天天氣真好，心情很愉快",
        "謝謝大家的幫助，非常感謝",
        "這個產品的品質還不錯"
    ],
    "負面情緒": [
        "我今天心情很糟糕",
        "這個決定讓我很失望",
        "感覺很沮喪，不知道該怎麼辦"
    ]
}

# 展示測試文本
for category, texts in test_texts.items():
    print(f"\n【{category}】")
    for i, text in enumerate(texts, 1):
        print(f"  {i}. {text}")

## 4. 單句解釋分析

對單個文本進行詳細的IG解釋分析

In [ ]:
# 選擇一個典型的毒性文本進行詳細分析
sample_text = "你這個垃圾，滾開！"
print(f"分析文本: {sample_text}")

# 執行IG解釋
result = explainer.explain_text(
    sample_text,
    n_steps=50,  # IG積分步數
    internal_batch_size=10
)

# 顯示預測結果
print(f"\n=== 預測結果 ===")
toxicity_labels = ['非毒性', '毒性', '嚴重毒性']
emotion_labels = ['正面', '中性', '負面']
bullying_labels = ['非霸凌', '騷擾', '威脅']

print(f"毒性預測: {toxicity_labels[result.toxicity_pred]} (信心度: {result.toxicity_prob:.3f})")
print(f"情緒預測: {emotion_labels[result.emotion_pred]} (信心度: {result.emotion_prob:.3f})")
print(f"霸凌預測: {bullying_labels[result.bullying_pred]} (信心度: {result.bullying_prob:.3f})")
print(f"收斂性檢查: {result.convergence_delta:.4f}")

# 顯示token級attribution
print(f"\n=== Token級Attribution分數 ===")
for i, token in enumerate(result.tokens):
    if token not in ['[CLS]', '[SEP]', '[PAD]'] and i < len(result.toxicity_attributions):
        clean_token = token.replace('##', '')
        print(f"{clean_token:>8} | "
              f"毒性: {result.toxicity_attributions[i]:>7.4f} | "
              f"情緒: {result.emotion_attributions[i]:>7.4f} | "
              f"霸凌: {result.bullying_attributions[i]:>7.4f}")

## 5. Attribution熱力圖可視化

創建token級重要度熱力圖

In [ ]:
# 創建毒性classification的熱力圖
fig_toxicity = create_attribution_heatmap(
    result, 
    task='toxicity',
    save_path='../reports/attribution_heatmap_toxicity.png',
    figsize=(14, 8)
)
plt.show()

# 創建情緒classification的熱力圖
fig_emotion = create_attribution_heatmap(
    result,
    task='emotion', 
    save_path='../reports/attribution_heatmap_emotion.png',
    figsize=(14, 8)
)
plt.show()

## 6. 批次文本對比分析

分析多個文本的attribution模式，識別共同特徵

In [ ]:
# 選擇不同類型的文本進行對比
comparison_texts = [
    "你這個垃圾，滾開！",      # 毒性
    "今天天氣真好，心情很愉快",    # 正面
    "我今天心情很糟糕",        # 負面但非毒性
    "去死吧，沒用的廢物"       # 嚴重毒性
]

print("執行批次分析...")
batch_results = []
for text in comparison_texts:
    result = explainer.explain_text(text)
    batch_results.append(result)
    print(f"✓ 完成: {text[:20]}...")

print(f"批次分析完成，共處理 {len(batch_results)} 個文本")

In [ ]:
# 創建對比可視化
fig, axes = plt.subplots(2, 2, figsize=(20, 16))
axes = axes.flatten()

for i, (result, text) in enumerate(zip(batch_results, comparison_texts)):
    # 過濾tokens和attributions
    tokens = []
    attrs = []
    for j, token in enumerate(result.tokens):
        if (token not in ['[CLS]', '[SEP]', '[PAD]'] and 
            j < len(result.toxicity_attributions)):
            clean_token = token.replace('##', '')
            if clean_token.strip():  # 過濾空token
                tokens.append(clean_token)
                attrs.append(result.toxicity_attributions[j])
    
    if not tokens:  # 如果沒有有效tokens，跳過
        continue
        
    # 正規化attribution分數
    attrs = np.array(attrs)
    if np.max(np.abs(attrs)) > 0:
        attrs = attrs / np.max(np.abs(attrs))
    
    # 設定顏色
    colors = ['red' if attr > 0 else 'blue' for attr in attrs]
    alphas = [abs(attr) for attr in attrs]
    
    # 繪製條形圖
    bars = axes[i].barh(range(len(tokens)), attrs, color=colors, alpha=0.7)
    
    # 設定標籤
    axes[i].set_yticks(range(len(tokens)))
    axes[i].set_yticklabels(tokens, fontsize=10)
    axes[i].set_xlabel('Toxicity Attribution Score', fontsize=12)
    axes[i].set_title(f'文本 {i+1}: {text}\n'
                     f'毒性預測: {["非毒性", "毒性", "嚴重毒性"][result.toxicity_pred]} '
                     f'({result.toxicity_prob:.3f})', fontsize=11)
    axes[i].axvline(x=0, color='black', linestyle='-', alpha=0.3)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/batch_attribution_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. 模型偏見分析

使用BiasAnalyzer檢測身份攻擊、辱罵詞等偏見模式

In [ ]:
# 準備包含各種偏見的測試文本
bias_test_texts = [
    # 身份相關
    "你們這些外地人就是素質差",
    "女生就是不適合做技術工作", 
    "老年人就是學不會新技術",
    "中國人都很聰明",
    "男性在這方面比較有優勢",
    
    # 辱罵詞相關
    "這個笨蛋什麼都不懂",
    "你真是個廢物",
    "別這麼蠢好嗎",
    "垃圾政策害死人",
    
    # 正常文本（對照組）
    "今天天氣很好",
    "這個方案需要改進",
    "感謝大家的支持",
    "我不太同意這個觀點",
    "希望情況能夠好轉"
]

print(f"準備分析 {len(bias_test_texts)} 個文本的偏見模式...")

# 執行偏見分析
bias_df = bias_analyzer.analyze_bias_patterns(
    bias_test_texts,
    top_k=10,
    output_csv='../reports/bias_analysis_report.csv'
)

print(f"偏見分析完成，發現 {len(bias_df)} 個偏見相關tokens")

In [ ]:
# 顯示偏見分析結果統計
if len(bias_df) > 0:
    print("=== 偏見分析統計結果 ===")
    
    # 按偏見類別統計
    bias_category_stats = bias_df['bias_category'].value_counts()
    print("\n偏見類別分布:")
    for category, count in bias_category_stats.items():
        print(f"  {category}: {count} 個tokens")
    
    # 按子類別統計
    bias_subcategory_stats = bias_df['bias_subcategory'].value_counts()
    print("\n偏見子類別分布:")
    for subcategory, count in bias_subcategory_stats.items():
        print(f"  {subcategory}: {count} 個tokens")
    
    # 顯示attribution分數最高的偏見詞
    print("\n=== Top-10 偏見相關詞彙 (按總重要度排序) ===")
    top_bias_tokens = bias_df.nlargest(10, 'total_importance')
    
    for idx, row in top_bias_tokens.iterrows():
        print(f"{row['token']:>8} | "
              f"類別: {row['bias_category']:>8} | "
              f"子類: {row['bias_subcategory']:>12} | "
              f"毒性attr: {row['toxicity_attribution']:>7.4f} | "
              f"總重要度: {row['total_importance']:>7.4f}")
else:
    print("未檢測到偏見模式（可能需要調整偏見詞庫或測試文本）")

## 8. 偏見模式可視化

視覺化不同類型偏見的attribution分布

In [ ]:
if len(bias_df) > 0:
    # 創建偏見分析可視化
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. 偏見類別分布
    bias_category_counts = bias_df['bias_category'].value_counts()
    axes[0, 0].pie(bias_category_counts.values, labels=bias_category_counts.index, autopct='%1.1f%%')
    axes[0, 0].set_title('偏見類別分布')
    
    # 2. Attribution分數分布（毒性）
    axes[0, 1].hist(bias_df['toxicity_attribution'], bins=20, alpha=0.7, color='red')
    axes[0, 1].set_title('偏見詞彙的毒性Attribution分布')
    axes[0, 1].set_xlabel('Toxicity Attribution Score')
    axes[0, 1].set_ylabel('Frequency')
    
    # 3. 不同偏見類別的attribution對比
    sns.boxplot(data=bias_df, x='bias_category', y='total_importance', ax=axes[1, 0])
    axes[1, 0].set_title('不同偏見類別的總重要度分布')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. 毒性預測概率 vs Attribution分數散點圖
    scatter = axes[1, 1].scatter(bias_df['toxicity_attribution'], 
                                bias_df['toxicity_prob'], 
                                c=bias_df['bias_category'].astype('category').cat.codes, 
                                alpha=0.7)
    axes[1, 1].set_xlabel('Toxicity Attribution Score')
    axes[1, 1].set_ylabel('Toxicity Prediction Probability')
    axes[1, 1].set_title('Attribution分數 vs 預測概率')
    
    plt.tight_layout()
    plt.savefig('../reports/bias_analysis_visualization.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("無偏見資料可視覺化")

## 9. 誤判案例診斷

分析模型的誤判案例，理解模型決策的問題

In [ ]:
# 準備一些可能導致誤判的邊界案例
edge_cases = [
    {
        "text": "這個垃圾桶需要清理了",  # 包含"垃圾"但非辱罵
        "expected_toxicity": 0,  # 預期非毒性
        "description": "包含負面詞彙但語境正常"
    },
    {
        "text": "你的想法很有創意，但我不太認同",  # 禮貌的不同意見
        "expected_toxicity": 0,
        "description": "禮貌表達不同意見"
    },
    {
        "text": "殺價成功，省了不少錢",  # "殺"字但非暴力
        "expected_toxicity": 0,
        "description": "包含暴力詞彙但語境無害"
    },
    {
        "text": "你說得對，我想太多了",  # 自我反省
        "expected_toxicity": 0,
        "description": "自我反省，非攻擊性"
    },
    {
        "text": "這個決定真的讓人很生氣",  # 強烈負面情緒但非辱罵
        "expected_toxicity": 0,
        "description": "強烈負面情緒但非人身攻擊"
    }
]

print("執行誤判案例診斷...")
misclassification_analysis = []

for case in edge_cases:
    result = explainer.explain_text(case["text"])
    
    # 判斷是否誤判（簡化：毒性分數>0.5視為毒性預測）
    is_predicted_toxic = result.toxicity_pred > 0
    is_expected_toxic = case["expected_toxicity"] > 0
    is_misclassified = is_predicted_toxic != is_expected_toxic
    
    analysis = {
        "text": case["text"],
        "description": case["description"],
        "expected_toxic": is_expected_toxic,
        "predicted_toxic": is_predicted_toxic,
        "is_misclassified": is_misclassified,
        "toxicity_prob": result.toxicity_prob,
        "result": result
    }
    
    misclassification_analysis.append(analysis)

print("誤判案例診斷完成")

In [ ]:
# 顯示誤判案例分析結果
print("=== 誤判案例診斷報告 ===")

misclassified_cases = [case for case in misclassification_analysis if case["is_misclassified"]]
correct_cases = [case for case in misclassification_analysis if not case["is_misclassified"]]

print(f"\n總測試案例: {len(misclassification_analysis)}")
print(f"誤判案例: {len(misclassified_cases)}")
print(f"正確分類: {len(correct_cases)}")
print(f"準確率: {len(correct_cases)/len(misclassification_analysis)*100:.1f}%")

if misclassified_cases:
    print("\n=== 誤判案例詳細分析 ===")
    for i, case in enumerate(misclassified_cases, 1):
        result = case["result"]
        print(f"\n誤判案例 {i}:")
        print(f"  文本: {case['text']}")
        print(f"  描述: {case['description']}")
        print(f"  預期: {'毒性' if case['expected_toxic'] else '非毒性'}")
        print(f"  預測: {'毒性' if case['predicted_toxic'] else '非毒性'} (概率: {case['toxicity_prob']:.3f})")
        
        # 分析關鍵詞attribution
        print(f"  關鍵token attribution:")
        for j, token in enumerate(result.tokens):
            if (token not in ['[CLS]', '[SEP]', '[PAD]'] and 
                j < len(result.toxicity_attributions) and
                abs(result.toxicity_attributions[j]) > 0.01):  # 只顯示重要的attribution
                clean_token = token.replace('##', '')
                print(f"    {clean_token}: {result.toxicity_attributions[j]:.4f}")
else:
    print("\n✓ 所有測試案例都被正確分類")

## 10. 生成完整解釋報告

匯總所有分析結果，生成完整的可解釋性報告

In [ ]:
# 收集所有分析過的結果
all_results = []
all_results.extend(batch_results)
all_results.extend([case["result"] for case in misclassification_analysis])

print(f"生成包含 {len(all_results)} 個樣本的完整解釋報告...")

# 儲存詳細的attribution報告
save_attribution_report(
    all_results,
    output_path='../reports/complete_attribution_report.csv',
    include_tokens=True
)

# 儲存摘要報告
save_attribution_report(
    all_results,
    output_path='../reports/attribution_summary_report.csv',
    include_tokens=False
)

print("報告生成完成！")
print("\n生成的檔案:")
print("  - ../reports/attribution_heatmap_toxicity.png: 毒性attribution熱力圖")
print("  - ../reports/attribution_heatmap_emotion.png: 情緒attribution熱力圖")
print("  - ../reports/batch_attribution_comparison.png: 批次對比圖")
print("  - ../reports/bias_analysis_report.csv: 偏見分析CSV報告")
print("  - ../reports/bias_analysis_visualization.png: 偏見分析可視化")
print("  - ../reports/complete_attribution_report.csv: 完整attribution報告")
print("  - ../reports/attribution_summary_report.csv: attribution摘要報告")

## 11. 總結與建議

根據IG分析結果，提供模型改進建議

In [ ]:
print("=== CyberPuppy IG可解釋性分析總結 ===")
print("\n1. 模型行為分析:")
print("   - 模型能夠識別明顯的辱罵詞彙和攻擊性語言")
print("   - 對於語境依賴的毒性判斷存在一定挑戰")
print("   - 某些無害的負面詞彙可能被過度解讀")

print("\n2. 偏見檢測結果:")
if len(bias_df) > 0:
    print(f"   - 檢測到 {len(bias_df)} 個偏見相關tokens")
    print(f"   - 主要偏見類型: {', '.join(bias_df['bias_category'].unique())}")
    print("   - 需要關注身份攻擊和刻板印象的檢測")
else:
    print("   - 當前測試集未檢測到明顯偏見模式")
    print("   - 建議使用更大的測試集進行偏見檢測")

print("\n3. 模型改進建議:")
print("   - 加強語境理解能力，減少對單個詞彙的過度依賴")
print("   - 增加邊界案例的訓練資料，提高區分能力")
print("   - 定期進行偏見檢測，確保公平性")
print("   - 考慮引入更多語義特徵，而非僅依賴詞彙層面")

print("\n4. 部署建議:")
print("   - 建立人工審核機制處理邊界案例")
print("   - 設定適當的信心度閾值，避免誤判")
print("   - 定期收集用戶反饋，持續改進模型")
print("   - 建立可解釋性監控，追蹤模型決策品質")

print("\n=== 分析完成 ===")